In [1]:
%matplotlib inline

In [2]:
import sys
import numpy as np
import os
import matplotlib.pyplot as plt


from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv2D, Convolution2D, MaxPooling2D, Flatten

Using TensorFlow backend.


In [3]:
train_data = np.load('workdir/train_tr_MFCC_0_D_A_norm_data.npy')
train_labels = np.load('workdir/train_tr_MFCC_0_D_A_norm_labels.npy')
val_data = np.load('workdir/train_va_MFCC_0_D_A_norm_data.npy')
val_labels = np.load('workdir/train_va_MFCC_0_D_A_norm_labels.npy')
test_data = np.load('workdir/test_MFCC_0_D_A_norm_data.npy')
test_labels = np.load('workdir/test_MFCC_0_D_A_norm_labels.npy')

print("Shape of training, validation and test data and corresponding labels")
for data, labels in [(train_data, train_labels), (val_data, val_labels), (test_data, test_labels)]:
    print(data.shape, labels.shape)

Shape of training, validation and test data and corresponding labels
(13, 3, 13, 1359661) (1359661,)
(13, 3, 13, 147731) (147731,)
(13, 3, 13, 1527014) (1527014,)


In [4]:
print(np.unique(train_labels))
print(len(np.unique(train_labels)))
nb_classes = len(np.unique(train_labels))
y_train_asd = np.array(train_labels) 
#y_test = np.array(y_test)
Y_train_asd = np_utils.to_categorical(y_train_asd, nb_classes)
#Y_test = np_utils.to_categorical(y_test, nb_classes)
print(Y_train_asd.shape)
print(train_labels[0])
print(Y_train_asd[0])

[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.
  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.  28.  29.
  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.  42.  43.  44.
  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.  56.  57.  58.  59.
  60.  61.  62.  63.]
64
(1359661, 64)
42.0
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


#### Change order of columns

In [5]:
print(train_data.shape)
X_train = np.transpose(train_data, (3, 0, 2, 1))
Y_train_num = train_labels
X_val = np.transpose(val_data, (3, 0, 2, 1))
Y_val_num = val_labels
X_test = np.transpose(test_data, (3, 0, 2, 1))
Y_test_num = test_labels

nb_classes = 64 #number of phonemes

Y_train = np_utils.to_categorical(Y_train_num, nb_classes)
Y_val = np_utils.to_categorical(Y_val_num, nb_classes)
Y_test = np_utils.to_categorical(Y_test_num, nb_classes)


print("Shape of re-ordered training, validation and test data and corresponding labels")
for data, labels in [(X_train, Y_train), (X_val, Y_val), (X_test, Y_test)]:
    print(data.shape, labels.shape)

(13, 3, 13, 1359661)
Shape of re-ordered training, validation and test data and corresponding labels
(1359661, 13, 13, 3) (1359661, 64)
(147731, 13, 13, 3) (147731, 64)
(1527014, 13, 13, 3) (1527014, 64)


In [6]:
model = Sequential()
model.add(Convolution2D(32, (3, 3), padding='same',
                        input_shape=(13, 13, 3), activation='relu'))
model.add(Convolution2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

#model.add(Convolution2D(64, (3, 3), padding='same', activation='relu'))
#model.add(Convolution2D(64, (3, 3), activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.1))
#model.add(Convolution2D(32, (3, 3), activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(28))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer="rmsprop", metrics=['accuracy'])

#history = model.fit(X_train, Y_train, batch_size=64, epochs=250, verbose=1, validation_data=(X_test, Y_test))
history = model.fit(X_train, Y_train, batch_size=64, epochs=5, verbose=2, validation_data=(X_val, Y_val))
loss_and_metrics = model.evaluate(X_test, Y_test)
print(loss_and_metrics)

Train on 1359661 samples, validate on 147731 samples
Epoch 1/5
122s - loss: 0.9559 - acc: 0.6945 - val_loss: 0.6868 - val_acc: 0.7720
Epoch 2/5
85s - loss: 0.8297 - acc: 0.7351 - val_loss: 0.6855 - val_acc: 0.7788
Epoch 3/5
85s - loss: 0.8777 - acc: 0.7274 - val_loss: 0.7242 - val_acc: 0.7705
Epoch 4/5
85s - loss: 0.9302 - acc: 0.7156 - val_loss: 0.7821 - val_acc: 0.7588
Epoch 5/5
84s - loss: 0.9778 - acc: 0.7040 - val_loss: 0.8633 - val_acc: 0.7475
1526848/1527014 [============================>.] - ETA: 0s   